https://www.kaggle.com/datasets/bohnacker/country-longitude-latitude?resource=download


In [824]:
import json

import numpy as np
import pandas as pd

In [825]:
MIN_BOOKS_THRESHOLD = 100

In [826]:
file_path = './data/hotel_bookings.csv'
bookings_df = pd.read_csv(file_path)

print(bookings_df.columns)

bookings_df.head()

Index(['Unnamed: 0', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'dia', 'tipo'],
      dtype='object')


,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,dia,tipo
0,3,Resort Hotel,0,7,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
1,4,Resort Hotel,0,13,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
2,5,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
3,6,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
4,7,Resort Hotel,0,0,2015,July,27,1,0,2,...,NaN,0,Transient,107.0,0,0,Check-Out,2015-07-03,2015-07-01,work


In [827]:
bookings_df.loc[bookings_df['country'] == 'CN', 'country'] = 'CHN'

In [828]:
grouped_countries = bookings_df.groupby(['country']).agg(
    total_books = ('is_canceled', 'count')
).reset_index()

grouped_countries = grouped_countries[grouped_countries['total_books'] >= MIN_BOOKS_THRESHOLD]

relevant_countries = grouped_countries['country']
num_books_before_clean = bookings_df.shape[0]

bookings_df = bookings_df[bookings_df['country'].isin(relevant_countries)]

num_books_after_clean = bookings_df.shape[0]

print(f'A total {num_books_before_clean-num_books_after_clean} of books have been cleaned')

A total 2841 of books have been cleaned


In [829]:
file_path = './data/countries.csv'
countries_df = pd.read_csv(file_path)


countries_df.head()

repeated = countries_df['ISO-ALPHA-3'].value_counts()[countries_df['ISO-ALPHA-3'].value_counts() > 1].index

for country in repeated.to_list():
    if country in bookings_df['country'].unique():
        print(countries_df[countries_df['ISO-ALPHA-3'] == country][['Country', 'ISO-ALPHA-3']])

unused_countries = ["Russian Federation", "China, People's Republic of", "United States", 
                    "United Kingdom of Great Britain and Northern Ireland", "West Germany", "Korea, Republic of (South)", "Korea, Republic of"]

countries_df = countries_df[~countries_df['Country'].isin(unused_countries)]

                        Country ISO-ALPHA-3
154  Korea, Republic of (South)         KOR
155                 South Korea         KOR
156          Korea, Republic of         KOR
                        Country ISO-ALPHA-3
53                        China         CHN
54  China, People's Republic of         CHN
                Country ISO-ALPHA-3
233  Russian Federation         RUS
234              Russia         RUS
           Country ISO-ALPHA-3
74  Czech Republic         CZE
75         Czechia         CZE
                                               Country ISO-ALPHA-3
107                                     United Kingdom         GBR
108  United Kingdom of Great Britain and Northern I...         GBR
                      Country ISO-ALPHA-3
286  United States of America         USA
287             United States         USA
         Country ISO-ALPHA-3
78       Germany         DEU
79  West Germany         DEU


In [830]:
country_to_iso = {}
iso_to_country = {}


for country in bookings_df['country'].unique():
    iso = countries_df[countries_df['ISO-ALPHA-3'] == country]['Country'].values
    if len(iso) == 0:
        print(country)
        continue
    country_to_iso[iso[0]] = country
    iso_to_country[country] = iso[0]
    
print(country_to_iso)
print(iso_to_country)

{'United Kingdom': 'GBR', 'Portugal': 'PRT', 'United States of America': 'USA', 'Spain': 'ESP', 'Ireland': 'IRL', 'France': 'FRA', 'Romania': 'ROU', 'Norway': 'NOR', 'Argentina': 'ARG', 'Poland': 'POL', 'Germany': 'DEU', 'Belgium': 'BEL', 'Switzerland': 'CHE', 'China': 'CHN', 'Greece': 'GRC', 'Italy': 'ITA', 'Netherlands': 'NLD', 'Denmark': 'DNK', 'Russia': 'RUS', 'Sweden': 'SWE', 'Australia': 'AUS', 'Czech Republic': 'CZE', 'Brazil': 'BRA', 'Finland': 'FIN', 'Luxembourg': 'LUX', 'India': 'IND', 'Morocco': 'MAR', 'Serbia': 'SRB', 'Austria': 'AUT', 'Turkey': 'TUR', 'Israel': 'ISR', 'Algeria': 'DZA', 'South Korea': 'KOR', 'Hungary': 'HUN', 'Croatia': 'HRV', 'Angola': 'AGO', 'Japan': 'JPN'}
{'GBR': 'United Kingdom', 'PRT': 'Portugal', 'USA': 'United States of America', 'ESP': 'Spain', 'IRL': 'Ireland', 'FRA': 'France', 'ROU': 'Romania', 'NOR': 'Norway', 'ARG': 'Argentina', 'POL': 'Poland', 'DEU': 'Germany', 'BEL': 'Belgium', 'CHE': 'Switzerland', 'CHN': 'China', 'GRC': 'Greece', 'ITA': 'I

In [831]:
bookings_df.head()

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,dia,tipo
0,3,Resort Hotel,0,7,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
1,4,Resort Hotel,0,13,2015,July,27,1,0,1,...,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01,work
2,5,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
3,6,Resort Hotel,0,14,2015,July,27,1,0,2,...,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01,work
4,7,Resort Hotel,0,0,2015,July,27,1,0,2,...,NaN,0,Transient,107.0,0,0,Check-Out,2015-07-03,2015-07-01,work


In [832]:
charts_info_dict = {}

In [833]:
num_canceled = sum(bookings_df['is_canceled'] == 1)
num_confirmed = sum(bookings_df['is_canceled'] == 0)
cancel_rate = num_canceled/(num_canceled+num_confirmed) * 100
print(f'There is a total of {num_canceled} of canceled reservations')
print(f'There is a total of {num_confirmed} of confirmed reservations')
print(f'The cancel rate is: {cancel_rate}')

charts_info_dict['chart1'] = {'confirmations': num_confirmed, 'cancelations': num_canceled}

There is a total of 43169 of canceled reservations
There is a total of 71381 of confirmed reservations
The cancel rate is: 37.68572675687473


# Chart 2: Mapa mundial

Per cada país cal tenir:
<ul>
  <li> localització
  <li> total reserves (confirmades/cancelades)
  <li> total reserves per tipus de grup (familia, parella, sol)
  <li> total reserves per lloc de reserva (online, no-online)
  <li> total reserves per antel·lació en mesos( 1, 2, 3, 4, 5, 6, 7+)
</ul>

In [834]:
cancel_summary = bookings_df.groupby(['country']).agg(
    total_books = ('is_canceled', 'count'),
    cancelations = ('is_canceled', 'sum')
).reset_index()

cancel_summary = cancel_summary.loc[cancel_summary['total_books'] >= MIN_BOOKS_THRESHOLD]

cancel_summary['confirmations'] = cancel_summary['total_books'] - cancel_summary['cancelations']
cancel_summary['cancelations_rate'] = cancel_summary['cancelations'] / cancel_summary['total_books'] * 100

cancel_summary = cancel_summary.sort_values(by='cancelations_rate', ascending=False)

cancel_summary

,country,total_books,cancelations,confirmations,cancelations_rate
30,PRT,47020,27326,19694,58.115695
0,AGO,355,205,150,57.746479
26,MAR,256,109,147,42.578125
24,KOR,133,55,78,41.353383
35,TUR,248,102,146,41.129032
32,RUS,623,239,384,38.362761
25,LUX,285,109,176,38.245614
5,BRA,2210,829,1381,37.511312
22,ITA,3749,1332,2417,35.529475
18,HUN,228,77,151,33.771930


In [835]:
top_5_countries_books = cancel_summary.sort_values(by='total_books', ascending=False).head()['country'].values
print(top_5_countries_books)

countries_over_mean_df = cancel_summary[cancel_summary['cancelations_rate'] > 37]
country_over_mean_names = [iso_to_country[country] for country in countries_over_mean_df['country'].values]
country_over_mean_rates = countries_over_mean_df['cancelations_rate'].round(2).values

countries_over_mean_info = list(map(lambda name_rate: f"{name_rate[0]}-{name_rate[1]}%", zip(country_over_mean_names, country_over_mean_rates)))
print(countries_over_mean_info)

['PRT' 'GBR' 'FRA' 'ESP' 'DEU']
['Portugal-58.12%', 'Angola-57.75%', 'Morocco-42.58%', 'South Korea-41.35%', 'Turkey-41.13%', 'Russia-38.36%', 'Luxembourg-38.25%', 'Brazil-37.51%']


In [836]:
charts_info_dict['chart5'] = countries_over_mean_info

In [837]:
charts_info_dict['chart2'] = {}

for _, country_data in cancel_summary.iterrows():
    country = country_data['country']
    charts_info_dict['chart2'][iso_to_country[country]] = {
        'total_books': country_data['total_books'],
        'cancelations': country_data['cancelations'],
        'confirmations': country_data['confirmations'],
        'cancelations_rate': country_data['cancelations_rate']
    }

In [838]:
conditions = [
    (bookings_df['lead_time'] > 180),
    (bookings_df['lead_time'] <= 180) & (bookings_df['lead_time'] > 150),
    (bookings_df['lead_time'] <= 150) & (bookings_df['lead_time'] > 120),
    (bookings_df['lead_time'] <= 120) & (bookings_df['lead_time'] > 90),
    (bookings_df['lead_time'] <= 90) & (bookings_df['lead_time'] > 60),
    (bookings_df['lead_time'] <= 60) & (bookings_df['lead_time'] > 30),
    (bookings_df['lead_time'] <= 30)
]

values = [7, 6, 5, 4, 3, 2, 1]

bookings_df['lead_months'] = np.select(conditions, values)
bookings_df['lead_months'].value_counts()

lead_months
1    35863
7    24233
2    16397
3    12239
4    10137
5     7950
6     7731
Name: count, dtype: int64

In [839]:
cancel_summary = bookings_df.groupby(['lead_months']).agg(
    total_books = ('is_canceled', 'count'),
    cancelations = ('is_canceled', 'sum')
).reset_index()

cancel_summary['confirmations'] = cancel_summary['total_books'] - cancel_summary['cancelations']
cancel_summary['cancelations_rate'] = cancel_summary['cancelations'] / cancel_summary['total_books'] * 100
cancel_summary = cancel_summary.sort_values(by='lead_months', ascending=False)

print(cancel_summary)

   lead_months  total_books  cancelations  confirmations  cancelations_rate
6            7        24233         13911          10322          57.405191
5            6         7731          3562           4169          46.074247
4            5         7950          3523           4427          44.314465
3            4        10137          4575           5562          45.131696
2            3        12239          4865           7374          39.749980
1            2        16397          5987          10410          36.512777
0            1        35863          6746          29117          18.810473


In [840]:
charts_info_dict['chart3'] = {}

for _, lead_month_data in cancel_summary.iterrows():
    charts_info_dict['chart3'][lead_month_data['lead_months']] = {
        'total_books': lead_month_data['total_books'],
        'cancelations': lead_month_data['cancelations'],
        'confirmations': lead_month_data['confirmations'],
        'cancelations_rate': lead_month_data['cancelations_rate']
    }

In [841]:
conditions = [
    ((bookings_df['babies'] > 0) | (bookings_df['children'] > 0)),
    (bookings_df['adults'] == 2),
    (bookings_df['adults'] > 2),
    (bookings_df['adults'] == 1)
]

values = ['Familia', 'Parella', 'Grups', 'Una Persona']

bookings_df['group_type'] = np.select(conditions, values)
bookings_df['group_type'].value_counts()

group_type
Parella        79138
Una Persona    20928
Familia         8956
Grups           5528
Name: count, dtype: int64

In [842]:
cancel_summary = bookings_df.groupby(['country', 'group_type']).agg(
    total_books = ('is_canceled', 'count'),
    cancelations = ('is_canceled', 'sum')
).reset_index()

cancel_summary['confirmations'] = cancel_summary['total_books'] - cancel_summary['cancelations']
cancel_summary['cancelations_rate'] = cancel_summary['cancelations'] / cancel_summary['total_books'] * 100

cancel_summary = cancel_summary[cancel_summary['country'].isin(top_5_countries_books)]
cancel_summary.head(12)

,country,group_type,total_books,cancelations,confirmations,cancelations_rate
36,DEU,Familia,364,92,272,25.274725
37,DEU,Grups,407,96,311,23.587224
38,DEU,Parella,5276,913,4363,17.304776
39,DEU,Una Persona,1199,117,1082,9.758132
48,ESP,Familia,1084,349,735,32.195572
49,ESP,Grups,550,179,371,32.545455
50,ESP,Parella,5814,1518,4296,26.109391
51,ESP,Una Persona,1040,130,910,12.500000
56,FRA,Familia,918,235,683,25.599129
57,FRA,Grups,723,221,502,30.567082


In [843]:
charts_info_dict['chart4'] = {}

for _, country_data in cancel_summary.iterrows():
    country = country_data['country']
    group_type = country_data['group_type']
    
    if country not in charts_info_dict['chart4']:
        charts_info_dict['chart4'][country] = {}
        
    charts_info_dict['chart4'][country][group_type] = {
        'total_books': country_data['total_books'],
        'cancelations': country_data['cancelations'],
        'confirmations': country_data['confirmations'],
        'cancelations_rate': country_data['cancelations_rate']
    }

In [844]:
adr_describe = bookings_df['adr'].describe()
print(adr_describe)

count    114550.000000
mean        103.415225
std          46.583792
min           0.260000
25%          70.440000
50%          95.000000
75%         126.000000
max         510.000000
Name: adr, dtype: float64


In [845]:
conditions = [
    (bookings_df['adr'] <= adr_describe['25%']),
    (bookings_df['adr'] > adr_describe['25%']) & (bookings_df['adr'] <= adr_describe['75%']),
    (bookings_df['adr'] > adr_describe['75%'])
]

values = ['Preu Baix', 'Preu Mig', 'Preu Alt']

bookings_df['adr_cat'] = np.select(conditions, values)
bookings_df['adr_cat'].value_counts()

adr_cat
Preu Mig     57584
Preu Baix    28643
Preu Alt     28323
Name: count, dtype: int64

In [ ]:
cancel_summary = bookings_df.groupby(['adr_cat']).agg(
    total_books = ('is_canceled', 'count'),
    cancelations = ('is_canceled', 'sum')
).reset_index()

cancel_summary['confirmations'] = cancel_summary['total_books'] - cancel_summary['cancelations']
cancel_summary['cancelations_rate'] = cancel_summary['cancelations'] / cancel_summary['total_books'] * 100

print(cancel_summary)

     adr_cat  total_books  cancelations  confirmations  cancelations_rate
0  Preu Baix        28643         10229          18414          35.712041
1   Preu Mig        57584         22024          35560          38.246735
2   Preu Alt        28323         10916          17407          38.541115


In [ ]:
cancel_summary = bookings_df.groupby(['country', 'adr_cat']).agg(
    total_books = ('is_canceled', 'count'),
    cancelations = ('is_canceled', 'sum')
).reset_index()

cancel_summary['confirmations'] = cancel_summary['total_books'] - cancel_summary['cancelations']
cancel_summary['cancelations_rate'] = cancel_summary['cancelations'] / cancel_summary['total_books'] * 100

order = pd.CategoricalDtype(categories=['Preu Baix', 'Preu Mig', 'Preu Alt'], ordered=True)
cancel_summary['adr_cat'] = cancel_summary['adr_cat'].astype(order)
cancel_summary = cancel_summary.sort_values('adr_cat').reset_index(drop=True)

cancel_summary = cancel_summary[cancel_summary['country'].isin(top_5_countries_books)]
cancel_summary.head(15)

,country,adr_cat,total_books,cancelations,confirmations,cancelations_rate
27,DEU,Preu Alt,1499,470,1029,31.354236
28,DEU,Preu Baix,951,46,905,4.837014
29,DEU,Preu Mig,4796,702,4094,14.637198
36,ESP,Preu Alt,3094,1060,2034,34.259858
37,ESP,Preu Baix,1825,135,1690,7.397260
38,ESP,Preu Mig,3569,981,2588,27.486691
42,FRA,Preu Alt,3068,823,2245,26.825293
43,FRA,Preu Baix,1508,75,1433,4.973475
44,FRA,Preu Mig,5783,1034,4749,17.879993
45,GBR,Preu Alt,2625,927,1698,35.314286


In [848]:
charts_info_dict['chart6'] = {}

for _, country_data in cancel_summary.iterrows():
    country = country_data['country']
    adr_cat = country_data['adr_cat']
    
    if country not in charts_info_dict['chart6']:
        charts_info_dict['chart6'][country] = {}
        
    charts_info_dict['chart6'][country][adr_cat] = {
        'total_books': country_data['total_books'],
        'cancelations': country_data['cancelations'],
        'confirmations': country_data['confirmations'],
        'cancelations_rate': country_data['cancelations_rate']
    }

In [849]:
output_json_path = '../public/data/hotel_bookings.json'
with open(output_json_path, 'w', encoding='utf-8') as file:
    json.dump(charts_info_dict, file, ensure_ascii=False, indent=4)